# Assignment - 2: Data Representation and Point Cloud Operations

Team Name: GSMF

Roll number: 2019102015 , 2019102031

# Instructions

- Code must be written in Python in Jupyter Notebooks. We highly recommend using anaconda distribution or at the minimum, virtual environments for this assignment.
- Save all your results in ```results/<question_number>/<sub_topic_number>/```
- The **References** section provides you with important resources to solve the assignment.
- Make sure your code is modular since you may need to reuse parts for future assignments.
- Answer the descriptive questions in your own words with context & clarity. Do not copy answers from online resources or lecture notes.
- The **deadline** for this assignment is on 26/09/2021 at 11:55pm. Please note that there will be no extensions.
- Plagiarism is **strictly prohibited**.

# Submission Instructions

1. Make sure your code runs without any errors after reinitializing the kernel and removing all saved variables.
2. After completing your code and saving your results, zip the folder with name as ``Team_<team_name>_MR2021_Assignment_<assignment_number>.zip``

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import open3d as o3d
import copy
import os

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


# Introduction to types of Transformations and Homogeneous coordinates

In robotics applications, it is inevitable to keep track of the frames of multiple objects/worlds. These frames can be transformations from one coordinate frame to the other. **Homogeneous coordinates** help in keeping track of various coordinate frames and allow performing composition of various transforms. We will first try to understand between types of transformations and their invariant properties.
1. What is the difference between Affine, Similarity, and Euclidean transform? What are the invariant properities of each type of transform?
2. Watch this [video](https://www.youtube.com/watch?v=PvEl63t-opM) to briefly understand homogeneous coordinates. What are points at infinity? What type of transformation can you apply to transform a point from infinity to a point that is not at infinity? 
3. Using homogeneous coordinates we can represent different types of transformation as point transforms vs. frame transforms. Concatenation of transforms (whether you post multiply transformation matrices or pre-multiply transformation matrices) depends on the problem and how you are viewing it. Try to understand the difference between frame vs. point transformations from this [video](https://youtu.be/Za7Sdegf8m8?t=1834). Let's assume that our camera and world frames are coinciding with each other. We need to estimate the camera to world **frame** transformation matrix after applying the transformations defined below in terms of $T_i$.We apply **frame** transform to move the camera in the world in the following order:
    1. $T_1$ from the camera coordinate frame.
    2. $T_2$ from the world coordinate frame.
    3. $T_3$ from the world coordinate frame.
    4. $T_4$ from the camera coordinate frame.
    5. $T_5$ from the camera coordinate frame.


### Solution 1
#### Affine Transformation:
Affine transformation is a type of transformation in which the laws such as parallelism and collinearity of points is preserved. After the transformation, all points lying on a line still lie on a line ad the ratio of distances between points are also preserved. ( Eg: The midpoint of a given line remains the same even after transformation.)

While an affine transformation preserves proportions on lines, it does not necessarily preserve angles or lengths.

#### Similarity Transformation:
Similarity transformations are transformation in which the shape of the object under transformation ,ie, the geometry , angles etc are preserved. This involves rotation , translation , reflection and scaling operations.

#### Euclidean Transformation:
Euclidean transformation is most rigid transformation of the above two. It involves only translation , rotation and reflection. In this transformation , since scaling is not involved, the size of the object is preserved.


### Solution 2

Homogeneous coordinates are projective coordinates of euclidean space represented in 4 diensional vectors. The last row has a scalar by which if we divide all the other rows, we get back the euclidean coordinates.If we fix this last row scalar to 0 , we can represnt infinitely far points in homogeneous coordinates.

Consider the following transformations possible in homogeneous coordinates.
<img src="./img/homogeneoustransforms.png" width="50%"  >

To bring a point at infinity, the scalr in the last row must be non-zero in the last row.
Consider the transformation matrix:
\begin{pmatrix}
t11 & t12 & t13 &t14 \\
t21 & t22 & t23 & t24 \\
t31 & t32 & t33 & t34 \\
t41 & t42 & t43 & t44 \\
\end{pmatrix}
And consider a point at infinity whose homogeneous coordinated is given as:
\begin{pmatrix}
x \\
y \\
z \\
0 \\
\end{pmatrix}

After the transformation, the scalar in the last row of homogeneous coordinates is given as:
          
                  t41*x + t42*y +t43*z + t44*(0)
In all the transformations except projective transsformation , 

                    (t41 ,t42 , t43 ) = (0,0,0)
While in projective tranformation , it is 

                    (t41 ,t42 , t43 ) = pT
So, we can bring point at infinity to a point that is not at infinity, we can use projective transformation.



### Solution 3

We premultiply the transformation matrix if we want to perform the transformation in the world coordinate frame while we post multiply if we want to perform the rotation in the camera's frame.

So the final transformation matrix is given as:
                                    
                                 T_res =  T3 x T2 x Ti x T1 x T4 x T5
                                   
                                   

# Visualise the Data

Point clouds are a collection of points that represent a 3D shape or feature. Each point has its own set of X, Y and Z coordinates and in some cases additional attributes. A popular way to obtain this is by photogrammetry, though here we will use LiDAR data.

LiDAR is a remote sensing process which collects measurements used to create 3D models and maps of objects and environments. Using ultraviolet, visible, or near-infrared light, LiDAR gauges spatial relationships and shapes by measuring the time it takes for signals to bounce off objects and return to the scanner.

1. Download the data from [here](https://iiitaphyd-my.sharepoint.com/:f:/g/personal/venkata_surya_students_iiit_ac_in/EnYAMaTVIhJItzKYqtahE30BRKB6p6UfHN3TyJzvo6Mw0g?e=PegWds). It contains the LIDAR sensor output and odometry information per frame.

    The .bin files contain the 3D point cloud captured by the LIDAR in this format - x, y, z, and reflectance. 

    The odometry information is given in the `odometry.txt` file, which is a 12 element vector. Reshape each of the first 77 rows to a 3x4 matrix to obtain the pose.
    

2. Obtain the point cloud from this and visualise for 1-2 frames.

In [2]:
### function to convert data read from .bin to pcd ###
def convert_bin_to_pcd(bin_file):
    
    points = bin_file.reshape((-1,4))[:,0:3]
    
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)
    
    return pcd


In [3]:
### function to visualize
def custom_visualize(geometry_list,front=None,lookat=None,up=None):
    
    vis = o3d.visualization.Visualizer()
    vis.create_window()
    
    for i in geometry_list:
        vis.add_geometry(i)
    
    if(front):
        o3d.visualization.ViewControl.set_front(vis.get_view_control(), front)
    if(lookat):
        o3d.visualization.ViewControl.set_lookat(vis.get_view_control(), lookat)
    if(up):
        o3d.visualization.ViewControl.set_up(vis.get_view_control(), up)
    
    vis.run()
    vis.destroy_window()

In [4]:
### 1. reshape obtain pose ###

temp_poses = np.loadtxt("./data/odometry.txt",dtype=np.float32)[:77]
temp_poses = temp_poses.reshape(77,3,4)
row_4 = np.array([0,0,0,1])
poses = [] 
for i in range(77):
    poses.append(np.vstack((temp_poses[i],row_4)))

### 2. obtain point cloud and visualize ### 
pcd_list = []
for i in range(77):
    temp = np.fromfile("./data/LiDAR/0000"+str(i+10)+".bin", dtype=np.float32)
    pcd_list.append(convert_bin_to_pcd(temp).uniform_down_sample(10))
    
i = np.random.randint(low=0,high=76,size=(2,))

custom_visualize([pcd_list[i[0]]])
custom_visualize([pcd_list[i[1]]])

# Transform 

The point cloud obtained is with respect to the LiDAR frame. The poses however, are in the camera frame. If we want to combine the point clouds from various frames, we need to bring them to the camera frame. 

1. Refer to the image below and apply the required transformation to the point cloud. 

2. Then, register all point clouds into a common reference frame and visualise it (Open3D). It is helpful to use homogeneous coordinates to keep track of the different frames.

3. Write a function to transform the registered point cloud from the world to the $i^{th}$ camera frame, wherein $i$ is the input to the function.

4. \[Bonus\] Move around in the registered point cloud using arrow keys like you would do in a game. For this you will have to regularly transform the entire registered world to your current camera frame and visualize repeatedly. You may choose to avoid visualizing points that are behind the camera in this case as they are not visible from the scene. You may also visualize points at a max depth to make the process easier.

![](./img/transform.png)

In [5]:
### 1. transform from lidar fram to camera frame ###
T_lidar_to_cam = np.array( [ [ 0 , -1 ,  0 , 0 ] ,
                             [ 0 ,  0 , -1 , 0 ] ,
                             [ 1 ,  0 ,  0 , 0 ] ,
                             [ 0 ,  0 ,  0 , 1 ] ] )

pcd_list_copy = copy.deepcopy(pcd_list)

for pcd in pcd_list_copy:
    pcd.transform(T_lidar_to_cam)

### 2. register all point clouds into a common reference frame and visualise ###
for i in range(77):
    pcd_list_copy[i].transform(poses[i])
    
custom_visualize(pcd_list_copy)

In [7]:
### 3. function to transform the registered point cloud from the world to the  𝑖𝑡ℎ  camera frame ###
def world_to_cam(i):
    
    Tmat = np.empty([4,4])
    
    Tmat[3,:] = [0,0,0,1]
    Tmat[:3,:3] = poses[i][:3,:3].T
    Tmat[:3,3] = -np.matmul(Tmat[:3,:3],poses[i][:3,3])
    
    return Tmat

# Occupancy Map

Occupancy grid maps are discrete fine grain grid maps. These maps can be either 2-D or 3-D. Each cell in the occupancy grid map contains information on the physical objects present in the corresponding space. Since these maps shed light on what parts of the environment are occupied, and what is not, they are really useful for path planning and navigation.

Occupancy grid maps are probabilistic in nature due to noisy measurements. Each cell can have three states: Occupied, unoccupied, and unknown. For the purpose of this assignment, you can ignore the unknown and work in a binary setting where 1 is occupied and 0 is unoccupied.

1. The task here is to create an occupancy map for each LiDAR scan. You do not need to apply bayesian update rules here, just keep it simple. 

2. Now, using the *registered* point cloud, generate occupancy maps for each frame. What difference do you expect to see between the two methods?

    You can mark a cell as occupied based on a threshold of how many different z values are there for a particular (x,y) cell.

In [34]:
def Lidar_to_Camera(points):
        Lidar_to_Camera = np.array([ [0,0,1] , [-1,0,0] , [0,-1,0] ]).T
        return np.matmul(Lidar_to_Camera,points.T)

In [35]:
poses = np.loadtxt("./data/odometry.txt",dtype=np.float32)[:77]
poses = poses.reshape(77,3,4)

In [36]:
def get_next_frame(i):
    list_pcd = []
    bin_pcd = np.fromfile("./data/LiDAR/0000"+str(i+10)+".bin", dtype=np.float32)
    points = bin_pcd.reshape((-1, 4))[:, 0:3]
    points = points[0:points.size:10]
    points = Lidar_to_Camera(points).T

    pose =  poses[i]
    row_4 = np.array([0,0,0,1])
    Tmat = np.vstack((pose,row_4))
    points = np.vstack((points.T,np.ones((1,points.shape[0]),dtype=float)))
    transformed_points = np.matmul(Tmat,points).T
    car_coordinate = np.matmul(Tmat,row_4).T
    return transformed_points[:,:3],car_coordinate

In [48]:
import matplotlib.pyplot as plt
from matplotlib import animation
from PIL import Image as im

grid_width  = 500
grid_height = 700

car_trajectory = np.empty((0,3))

occupancy_grid = np.zeros((grid_width,grid_height))
accumulation_grid = np.zeros((grid_width,grid_height))
free_grid = np.zeros((grid_width,grid_height))  
occupied_image = np.zeros((occupancy_grid.shape[0],occupancy_grid.shape[1],3))
free_image = np.zeros((occupancy_grid.shape[0],occupancy_grid.shape[1],3))
for k in range(77):
    points,car_coordinate = get_next_frame(k)
    car_trajectory = np.vstack((car_trajectory,car_coordinate[:-1]))
    for point in points:
        if(point[1] > 0.5):
            free_grid[int(2*point[2]+grid_width/2)][int(2*point[0]+grid_height/2)] +=1 
            continue
        accumulation_grid[int(2*point[2]+grid_width/2)][int(2*point[0]+grid_height/2)] +=1 

    threshold = 1

    for i in range(grid_width):
        for j in range(grid_height):
            if(accumulation_grid[i][j]>=threshold):
                occupied_image[i][j] = np.array([255,255,255])
                free_image[i][j] = np.array([0,0,0])
            else:
                occupied_image[i][j] = np.array([0,0,0])
                free_image[i][j] = np.array([255,255,255])
                
    for point in car_trajectory:
        occupied_image[int(2*point[2]+grid_width/2)][int(2*point[0]+grid_height/2)] =  np.array([255,0,0])
        
# This directory has been moved so the below comment has been commented

#     data = im.fromarray(occupied_image.astype(np.uint8))
#     data = data.convert("RGB")
#     data.save('Anim/obstacle'+str(k)+'.png')


for point in car_trajectory:
        free_image[int(2*point[2]+grid_width/2)][int(2*point[0]+grid_height/2)] =  np.array([255,0,0])
    

# print(occupied_image.shape)
data = im.fromarray(occupied_image.astype(np.uint8))
data = data.convert("RGB")
data.save('obstacle.png')
            
data = im.fromarray(free_image.astype(np.uint8))
data = data.convert("RGB")
data.save('free_space.png')


### Estimating Trajectory of the Vehicle

In [ ]:
from mpl_toolkits import mplot3d
plist = np.empty((0,3))
for i in range(77):    
    poses = np.loadtxt("./data/odometry.txt",dtype=np.float32)[:77]
    poses = poses.reshape(77,3,4)
    P = poses[i][:,3].T
    plist  = np.vstack((plist,P))

fig = plt.figure()
 
plt.scatter(plist[:, 2],plist[:,0])
plt.show()